In [1]:
import keras

Using TensorFlow backend.


In [2]:
from keras.applications.vgg16 import VGG16
from keras.layers import Input, GlobalAveragePooling2D, Dense
from keras.models import Model
from keras.optimizers import SGD
import numpy as np
import pickle as pkl
import datetime

# Load Cifar10 dataset

In [20]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

In [4]:
print('The training set has {0} samples with dimentions {1}.'.format(x_train.shape[0], x_train.shape[1:]))
print('The test set has {0} samples with dimentions {1}.'.format(x_test.shape[0], x_test.shape[1:]))

The training set has 50000 samples with dimentions (32, 32, 3).
The test set has 10000 samples with dimentions (32, 32, 3).


In [26]:
NUM_CLASSES = 10
IMG_ROWS, IMG_COLS = 32, 32
BATCH_SIZE = 512
EPOCHS = 20
VAL_SPLIT = .2
LEARNING_RATE = 1e-2
SGD_MOMENTUM = .9

# Preprocess data

In [21]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [22]:
x_train -= np.mean(x_train)
x_test -= np.mean(x_test)

In [23]:
x_train /= np.std(x_train)
x_test /= np.std(x_test)

In [24]:
y_train = keras.utils.to_categorical(y_train, NUM_CLASSES)
y_test = keras.utils.to_categorical(y_test, NUM_CLASSES)

In [ ]:
val_split = int((1-VAL_SPLIT)*len(x_train))
x_train , x_val = x_train[:val_split], x_train[val_split:]
y_train, y_val = y_train[:val_split], y_train[val_split:]

# Build model

### Clear session from previou

In [11]:
input_tensor = Input(shape=(IMG_ROWS, IMG_COLS, 3))

In [12]:
base_model = VGG16(include_top=False, input_tensor=input_tensor)

In [14]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

In [15]:
vgg16 = Model(inputs=base_model.input, outputs=predictions)

In [16]:
vgg16.compile(optimizer=SGD(lr=LEARNING_RATE, momentum=SGD_MOMENTUM), loss='categorical_crossentropy', metrics=['accuracy'])

# Train model

In [27]:
now = datetime.datetime.now()
tensorflow_log_dir = "Graph/" + now.strftime("%Y%m%d-%H%M%S") + "/"

In [28]:
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=tensorflow_log_dir, histogram_freq=0,  
          write_graph=False, write_images=False)

In [ ]:
vgg16.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, callbacks=[tensorboard_callback], verbose=1)

Epoch 1/10
50000/50000 [==============================] - 45s 898us/step - loss: 0.2508 - acc: 0.9142
Epoch 2/10
50000/50000 [==============================] - 35s 705us/step - loss: 0.2108 - acc: 0.9291
Epoch 3/10
50000/50000 [==============================] - 35s 702us/step - loss: 0.1694 - acc: 0.9430
Epoch 4/10
50000/50000 [==============================] - 35s 699us/step - loss: 0.1592 - acc: 0.9511
Epoch 5/10
50000/50000 [==============================] - 35s 702us/step - loss: 0.1213 - acc: 0.9621
Epoch 6/10
50000/50000 [==============================] - 35s 702us/step - loss: 0.1019 - acc: 0.9706
Epoch 7/10
50000/50000 [==============================] - 35s 701us/step - loss: 0.1113 - acc: 0.9695
Epoch 8/10
50000/50000 [==============================] - 35s 701us/step - loss: 0.1120 - acc: 0.9721
Epoch 9/10
36352/50000 [====================>.........] - ETA: 9s - loss: 0.0893 - acc: 0.9778

In [27]:
vgg16.evaluate(x_test, y_test, batch_size=BATCH_SIZE, verbose=1)

10000/10000 [==============================] - 3s 271us/step


[0.6099598478317261, 0.8641]